<a href="https://colab.research.google.com/github/kate-ev/deep-learning/blob/main/rock_paper_scissors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **1.lielais mājasdarbs - "Akmens-šķēres-papīrītis"**

In [ ]:
# Autore: Jekaterina Jevtejeva
# St.apl.numurs: jj19021

# Spēles gājiena hot-kodējums, definēts pēc uzdevuma nosacījumiem
rock = [1, 0, 0] # akmens
scissors = [0, 1, 0] # šķēres
paper = [0, 0, 1] # papīrītis

# Spēles rezultāta kodējums, definēts pēc uzdevuma nosacījumiem
fst_win = [1, 0] # pirmais spēlētājs uzvar
snd_win = [0, 1] # otrais spēlētājs uzvar
undecided = [1, 1] # neizšķirts

# Definē visas iespējamās gājienu kombinācijas
input = [[rock, rock],
        [rock, scissors],
        [rock, paper],
        [scissors, rock],
        [scissors, scissors],
        [scissors, paper],
        [paper, rock],
        [paper, scissors],
        [paper, paper]]

# Definē spēles rezultātus ieejas datu kombināciju secībā
expected = [undecided,
            fst_win,
            snd_win,
            snd_win,
            undecided,
            fst_win,
            fst_win,
            snd_win,
            undecided]

import torch
from torch import nn

# Tīkla modulis
class RockPaperScissors(nn.Module):

  def __init__(self, batch_size, num_vectors, in_features, out_features):
    super().__init__()
    # Vērtības, kas ir nepieciešamas datu pārveidojumiem
    self.batch_size = batch_size
    self.flattened = num_vectors * in_features
    # Slāņi
    self.linear = nn.Linear(self.flattened, self.flattened)
    self.act_fn = nn.ReLU()
    self.linear2 = nn.Linear(self.flattened, out_features)
    self.act_fn2 = nn.Sigmoid()

  def forward(self, x):
    # Pārveido ievaddatus no 3D [9,2,3] uz 2D [9,6], izmantojot izrēķināto vērtību flattened
    x = torch.reshape(x, [self.batch_size, self.flattened])
    # Secīgi pielieto slāņus
    x = self.linear(x)
    x = self.act_fn(x)
    x = self.linear2(x)
    x = self.act_fn2(x)
    return x

import torch.utils.data as data

# Dataset realizācija
class RPSDataset(data.Dataset):

    def __init__(self):
        super().__init__()
        # Veido tenzorus no datiem
        training_data = torch.tensor(input)
        self.training_data = torch.tensor(training_data, dtype=torch.float32)
        self.training_expected = torch.tensor(expected, dtype=torch.float32)

    def __len__(self):
        return 9

    def __getitem__(self, idx):
        data_point = self.training_data[idx]
        data_label = self.training_expected[idx]
        return data_point, data_label

dataset = RPSDataset()
data_loader = data.DataLoader(dataset, batch_size=9, shuffle=False)

# batch_size - ieejas variantu daudzums
# num_vectors - no cik vektoriem sastāv katrs variants
# in_features - cik vērtības ir katrā vektorā
# out_features - cik vērtībām jābūt izejas vektorā
model = RockPaperScissors(batch_size=9, num_vectors=2, in_features=3, out_features=2)
model.train()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
lossfunction = nn.MSELoss()

# Modeļa apmācība
for epoch in range(15000):
  for data_inputs, data_labels in data_loader:
    result = model(data_inputs)

    # Izrēķina zaudējumfuknciju
    loss = lossfunction(result, data_labels)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()

    # Izmaina parametrus
    optimizer.step()

# Autore: Jekaterina Jevtejeva
# St.apl.numurs: jj19021

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:72: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Rezultāti

In [ ]:
# Autore: Jekaterina Jevtejeva
# St.apl.numurs: jj19021

# Pielieto apmācīto modeli visiem ievaddatiem
final = model(dataset.training_data)

print("Pielietojot apmācīto modeli visiem ievaddatiem: \n", final, "\n")

# Rezultātu izvades formatēšana
# Saraksts ar modeļa apmācības rezultātu vērtībām, noapaļojot līdz 4 decimālzīmēm
res_list = [[round(float(i), 4) for i in nested] for nested in final.tolist()]

# Saraksts ar modeļa apmācības rezultātu vērtībām, noapaļojot no float uz int
res_list_rounded = [[round(float(i)) for i in nested] for nested in res_list]

print("\n".join("Ievads: {} Sagaidāmais: {} Iegūtais: {} Noapaļojot: {}".format(w, x, y, z) for w, x, y, z in zip(input, expected, res_list, res_list_rounded)))

# Autore: Jekaterina Jevtejeva
# St.apl.numurs: jj19021